In [80]:
""" Importação de bibliotecas e constantes """

# Importar bibliotecas
# import pandas as pd
from openpyxl import load_workbook
from openpyxl import Workbook


In [82]:
""" Constantes """

FILE_PATH = 'B:/timetabling-UENF/Files/data/Processed Data/Tabelas/CS Grids - 2019.1-2024.1.xlsx'
times = [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
days = ["SEG", "TER", "QUA", "QUI", "SEX"]

In [83]:
""" Processar Tabela
    This receives an openpyxl worksheet gotten from an Excel file and processes it.
    
"""

# def sheetProcessScalable(worksheet):
#     for row in worksheet.iter_rows():
#         for cell in row:
#             message = ""
#             message += f"Cell {cell.coordinate} "
#             if cell.coordinate not in worksheet.merged_cells:
#                 message += "not "
#             else:
#                 message += "    "
#             message += "merged\t"
#             message += ": "
#             message += f"{cell.value}"
#             print(message)

In [87]:
""" Semi manual sheet processing
- set headers and hours
classTimes = [
  {id: 1, horaInicio: 8, duracao: 2, sala: "", dia: "SEG", info: ""},
]
"""

def processInfo(info):
  """ 
  expected input: info = "3-Est. Dados I (Tang-25/Bcct)"
  expected output:
  message = {
    expectedSemester: 3,
    subject: "Est. Dados I",
    professor: "Tang",
    expectedDemand: 25,
    room: "Bcct"
  }
  """
  message = {}
  
  parts = info.split("-")
  message["expectedSemester"] = int(parts[0].strip())
  
  subject_professor = parts[1].split("(")
  message["subject"] = subject_professor[0].strip()
  
  professor_demand_room = subject_professor[1].split("/")
  professor_demand = professor_demand_room[0].split("-")
  message["professor"] = professor_demand[0].strip()
  message["expectedDemand"] = int(professor_demand[1].strip())
  
  message["room"] = professor_demand_room[1].replace(")", "").strip()
  
  return message

def processCell(cell):
    if cell.value is not None:
        infoList = cell.value.strip().split("\n")
        infoList = [subject for subject in infoList if subject != ""]
        for info in infoList:
            message = {}
            message["duration"] = 2
            message["startHour"] = times[cell.row - 2]
            message["day"] = days[cell.column - 2]
            
            processedInfo = processInfo(info)
            print(processedInfo)
  

def sheetProcess(worksheet):
    for row in worksheet.iter_rows():
        for cell in row:
          if cell.row > 1 and cell.column > 1:
            processCell(cell)

In [85]:
""" pre processar arquivo
- Ler arquivo XLSX de um caminho armazenado em uma constante
- Iterar sobre todas as planilhas do arquivo
"""

# Carregar o arquivo Excel
workbook = load_workbook(FILE_PATH)

# Processar o arquivo

processOnce = True

for sheet in workbook.sheetnames:
    worksheet = workbook[sheet]
    if processOnce:
        processOnce = False
        sheetProcess(worksheet)

IndexError: list index out of range